<a href="https://www.kaggle.com/code/quctngngvng/sentence-trainformer-wiki?scriptVersionId=145199658" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **SEARCH CONTEXT WITH WIKIPEDIA**

In [ ]:
# installing offline dependencies
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/datasets-2.14.3-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

In [ ]:
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
from __future__ import annotations

from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")


from dataclasses import dataclass
from typing import Optional, Union

import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader

In [ ]:
SIM_MODEL = '/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 16

WIKI_PATH = "/kaggle/input/wikipedia-20230701"
wiki_files = os.listdir(WIKI_PATH)
## Parameter to determine how many relevant sentences to include \ Số lượng câu liên quan
NUM_SENTENCES_INCLUDE = 20

* ctypes library to load the libc.so.6 shared library, which is the C library on Linux systems
* _ = gc.collect()  : Forces an immediate garbage collection. This function collects and frees memory that is no longer needed.

In [ ]:
trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
# trn = pd.read_csv("/kaggle/input/15k-high-quality-examples/5900_examples.csv")
# trn.drop(columns = 'id')
trn = trn.fillna(" ")
trn.head(4)


In [ ]:
model = SentenceTransformer(SIM_MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half()

**Build index falatL2**

* Faiss (Facebook AI Similarity Search) để tìm kiếm hàng xóm gần nhất hiệu quả trong không gian chiều cao.
* IndexFlat  là một lớp hoặc khái niệm thường được liên kết với các thư viện và khung để tìm kiếm hàng xóm gần đúng gần nhất. Nó thường được sử dụng trong bối cảnh học máy và các tác vụ truy xuất thông tin, nơi bạn cần tìm các điểm dữ liệu gần hoặc tương tự với một điểm truy vấn nhất định một cách hiệu quả.

In [ ]:
# pip install faiss-cpu


In [ ]:
# import numpy as np
# import faiss

# # Sample data (you should replace this with your actual dataset)
# data = np.random.rand(100, 64).astype(np.float32)

# # Create an IndexFlatL2 index (L2 distance is commonly used for Euclidean distance)
# dimension = data.shape[1]
# index = faiss.IndexFlatL2(dimension)

# # Add your data to the index
# index.add(data)

**Load data wikipedia indexFlat**

In [ ]:
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

In [ ]:
prompt_embeddings = model.encode(sentences = trn.prompt.values, batch_size= BATCH_SIZE, show_progress_bar = True,convert_to_tensor=True, device=DEVICE , normalize_embeddings =True)
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
_ = gc.collect()

* .detach() : Phương pháp này được sử dụng để tạo một tensor mới chia sẻ cùng dữ liệu nhưng không được kết nối với biểu đồ tính toán. Nói cách khác, nó tách tensor khỏi mọi tính toán độ dốc. Điều này thường hữu ích khi bạn muốn chuyển đổi một tensor thành mảng NumPy vì bạn không cần độ dốc khi làm việc với kết quả cuối cùng.

In [ ]:
## Get the top 3 pages that are likely to contain the topic of interest
search_score, search_index = sentence_index.search(prompt_embeddings, 5)

* libc.malloc_trim(0) Trong C, thường được sử dụng để trả lại bộ nhớ cho hệ thống, cung cấp bộ nhớ cho các tiến trình khác khi bạn đang sử dụng cấp phát bộ nhớ động với các hàm như . Đối số được truyền cho biết rằng bạn muốn giải phóng càng nhiều bộ nhớ trống càng tốt.

In [ ]:
## Save memory - delete sentence_index since it is no longer necessary
del sentence_index
del prompt_embeddings
_ = gc.collect()
libc.malloc_trim(0)

In [ ]:
df = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
                     columns=['id', 'file'])

In [ ]:
# df.loc[search_index[0]].copy()   lấy list row theo search_index to index df
print(search_index[0].shape)
x = df.loc[search_index[0]].copy()
print(x)

In [ ]:
## Get the article and associated file location using the index
wikipedia_file_data = []

for i ,(scr,idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    scr_idx = idx # list id
    _df = df.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)

In [ ]:
wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
print(wikipedia_file_data.shape)

*sort datafame *
* drop_duplicates()  xóa trùng lặp columns

In [ ]:
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)
del df
_ = gc.collect()
libc.malloc_trim(0)

In [ ]:
wikipedia_file_data.head(5)

In [ ]:
wikipedia_file_data.file.unique()

In [ ]:
## Get the full text data
"""
    for i in list title wikipedia parquet(a,b,c,..z)
    _id - list id in title parquet (a/b/c/....)
    read parquet file in list and read [id,text] and 
    get all text in _id ||  _df_temp = _df[_df['id'].isin(_id)].copy()
    
    Giải phóng dữ liệu sau mỗi lần đọc file parquet
"""
wiki_text_data = []
for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])
    _df_temp = _df[_df['id'].isin(_id)].copy()
    del _df
    _ = gc.collect()
    libc.malloc_trim(0)
    wiki_text_data.append(_df_temp)
wiki_text_data =  pd.concat(wiki_text_data).reset_index(drop=True)
_ = gc.collect()

**process_documents**
input 
1. sectionize_documents  Đọc lân lượt dữ liệu 
* input document_id and text :  Full document
* sectionize_documents  function sử dụng trả về document_id , text and offset (Start,end) độ dài của text
* sentencize  function là tách document thành list tupe chứa danh sách (stat , end) tách văn bản thành danh sách các câu lẻ và lưu số chỉ vị trí đầu và cuối của câu


OUTPUT :  datafame chưa các hàng chứa từng câu đơn in text document và document_id and offset

In [ ]:
def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df




def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the 
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    
    offset 0 len(documents)   : Độ dài của văn bản

    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df
    
    
def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    
    bf.text_to_sentences_and_offsets(document)
    import blingfire as bf
    text_to_sentences_and_offsets  tách các câu đơn từ doccumente -> return list tuper(start,end)  
    
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    
    Đầu ra tách nhỏ list tuper(start,end )  trả về datafame row từng tupe in list 
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

In [ ]:
# test = sectionize_documents(documents =wiki_text_data.text.values, document_ids=wiki_text_data.id.values, disable_progress_bar =True)

In [ ]:
## Parse documents into sentences
processed_wiki_text_data = process_documents(wiki_text_data.text.values, wiki_text_data.id.values,disable_progress_bar = True)

In [ ]:
## Get embeddings of the wiki text data
wiki_data_embeddings = model.encode(processed_wiki_text_data.text,
                                    batch_size=BATCH_SIZE,
                                    device=DEVICE,
                                    show_progress_bar=True,
                                    convert_to_tensor=True,
                                    normalize_embeddings=True)#.half()
wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()
_ = gc.collect()

In [ ]:
## Combine all answers
# trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)
trn['answer_all'] = trn.apply(lambda x: " ".join([str(x['A']), str(x['B']), str(x['C']), str(x['D']), str(x['E'])]), axis=1)


## Search using the prompt and answers to guide the search
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']
# trn['prompt_answer_stem'] = trn['prompt']

In [ ]:
trn

In [ ]:
question_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
question_embeddings = question_embeddings.detach().cpu().numpy()

In [ ]:
processed_wiki_text_data.shape   # doc_id   text  offset

In [ ]:
wiki_text_data.shape # id and text

In [ ]:
# trn.to_csv("trn.csv", index=False)

In [ ]:
wikipedia_file_data.head(3) # id prompt_id  and file

In [ ]:
# wikipedia_file_data.to_csv("wikipedia_file_data.csv", index=False)

In [ ]:
processed_wiki_text_data

In [ ]:
# processed_wiki_text_data.to_csv("processed_wiki_text_data.csv", index=False)

In [ ]:
processed_wiki_text_data

In [ ]:


## List containing just Context
contexts = []


for r in tqdm(trn.itertuples(), total=len(trn)):
    prompt_id = r.Index
    
    # list id_document  with prompt_id = index (len = 5) -> id  doccument
    list_file_data_with_prompt_id = wikipedia_file_data[wikipedia_file_data['prompt_id']==prompt_id]['id'].values
    
    # đánh dấu row có id document in list_prompt_id
    list_text_in_document =  processed_wiki_text_data['document_id'].isin(list_file_data_with_prompt_id)
    # read data index with bool = True
    prompt_indices = processed_wiki_text_data[list_text_in_document].index.values
    if prompt_indices.shape[0] > 0: 
        # create store Flat
        prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
        # read index embedding and add faiss store flat ->  bộ dữ liệu ngữ cảnh
        prompt_index.add(wiki_data_embeddings[prompt_indices])
        context = ""
        score, index = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
        for _s, _i in zip(score[prompt_id], index[prompt_id]):
            context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + " "
    contexts.append(context)   

In [ ]:
trn['context'] = contexts

In [ ]:
# trn[["prompt", "context", "A", "B", "C", "D", "E"]].to_csv("./test_context.csv", index=False)

In [ ]:
del processed_wiki_text_data
del wikipedia_file_data
del wiki_data_embeddings
del contexts
_ = gc.collect()
libc.malloc_trim(0)

In [ ]:
trn = trn[["prompt", "context", "A", "B", "C", "D", "E"]]

# **PREDICT TEST AND SUBMISSION**

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoTokenizer
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

from typing import Optional, Union

In [ ]:
MODEL_NAME = '/kaggle/input/weight-2-10-debeta-v1-2-384/weigh_2_10_v1_2_384'
path_test = '/kaggle/working/test_context.csv'
MAX_INPUT = 384

In [ ]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    first_sentence = [ "[CLS] " + example['context'] ] * 5
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first', 
                                  max_length=MAX_INPUT, add_special_tokens=False)
#     tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
#         label_name = 'label' if 'label' in features[0].keys() else 'labels'
#         labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
#         batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
# df_test = trn
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# dataset_test = Dataset.from_pandas(df_test)

In [ ]:
tokenized_dataset_test = dataset_test.map(preprocess, remove_columns=['prompt', 'context', 'A', 'B', 'C', 'D', 'E'])

In [ ]:
model = AutoModelForMultipleChoice.from_pretrained(MODEL_NAME)
trainer = Trainer(model=model)

In [ ]:
model

In [ ]:
pip install torchviz torchview


In [ ]:
from torchview import draw_graph
from torchviz import make_dot

In [ ]:
prompt = "What's the capital of France?"
choices = ["Paris", "London", "Berlin", "Madrid",'VietNam']
# input_ids = tokenizer(prompt, choices, padding=True, truncation=True, return_tensors="pt")
encoding = tokenizer([prompt, prompt,prompt,prompt,prompt], [choices[0], choices[1],choices[2],choices[3],choices[4]], return_tensors="pt", padding=True,max_length=384)

In [ ]:
encoding['input_ids'].shape,encoding['token_type_ids'].shape,encoding['attention_mask'].shape

In [ ]:
encoding['input_ids'] = encoding['input_ids'].unsqueeze(0)
encoding['token_type_ids'] = encoding['token_type_ids'].unsqueeze(0)
encoding['attention_mask'] = encoding['attention_mask'].unsqueeze(0)

In [ ]:
model = model.to('cuda')
encoding = encoding.to('cuda')

In [ ]:
outputs = model(**encoding)

In [ ]:
# Create a visualization of the computation graph
make_dot(outputs.logits, params=dict(model.named_parameters())).render("multichoice_model_graph", format="png",)


In [ ]:
encoding['input_ids'][0]

In [ ]:
model_graph = draw_graph(model, input_data=encoding,graph_name='microsoft_Deberta_multichoice',
        roll=True,expand_nested = True,hide_inner_tensors=True,
        hide_module_functions=True,
        save_graph = True,
        filename='microsoft_Deberta_multichoice' ,
        directory ='/kaggle/working/' )

model_graph.visual_graph

In [ ]:
# test deberta
from transformers import AutoTokenizer, DebertaV2Model
import torch

tokenizer_2 = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge")
model_2 = DebertaV2Model.from_pretrained("microsoft/deberta-v2-xlarge")

inputs = tokenizer_2("Hello, my dog is cute", return_tensors="pt")
outputs2 = model_2(**inputs)


In [ ]:
# Create a visualization of the computation graph
make_dot(outputs2.last_hidden_state ,params=dict(model_2.named_parameters())).render("deberta_model_graph", format="png")


In [ ]:
from torchviz import make_dot
make_dot(output, params=dict(model.named_parameters())).render("simple_nn", format="png")

In [ ]:
import graphviz
graphviz.set_jupyter_format('png')

In [ ]:
model_graph = draw_graph(model_2, input_data=inputs,graph_name='microsoft_Deberta',
        roll=True,expand_nested = True,hide_inner_tensors=True,
        hide_module_functions=True,
        save_graph = True,
        filename='microsoft_Deberta' ,
        directory ='/kaggle/working/' )

model_graph.visual_graph

In [ ]:
test_predictions = trainer.predict(tokenized_dataset_test).predictions
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]

In [ ]:
test_predictions[0]

In [ ]:
df_submiss = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/sample_submission.csv')

In [ ]:
df_submiss['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

In [ ]:
df_submiss[["id", "prediction"]].to_csv("submission.csv", index=False)

In [ ]:
# score1 = 0
# score2 = 0
# for i,j in zip(df_submiss['answer'],df_submiss['prediction']):
#     if i == j[0]:
#         score1+=1
#         score2+=1
#     elif i == j[1]:
#         score2+=1/2
#     else:
#         score2+=1/3
# print(f"@MAP1 = {score1/200},@MAP3 = {score2/200}")
                

    